<a href="https://colab.research.google.com/github/myz21/BTK_Datathon/blob/main/BTK_Datathon24_rmse6_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Gerekli Kütüphanelerin Yüklenmesi

In [ ]:
#yeni kod
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
#from xgboost import XGBRegressor
#from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
import joblib
import warnings
warnings.filterwarnings("ignore")

## Eğitim Veri Setinin İşlenmesi

In [ ]:
#yeni kod
old = pd.read_csv('./sample_data/train.csv', low_memory=False)
old

,Basvuru Yili,Degerlendirme Puani,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,...,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",id
0,2014,52.0,Erkek,4/6/1994,"Altindag, Ankara",Ankara,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Evet,...,Diğer,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,0
1,2014,30.0,Erkek,6/11/1993,Üsküdar,İstanbul,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Hayır,...,0,Hayır,NaN,Evet,Evet,Bilkent Cyberparkta bir şirkette Türkiye nin i...,NaN,NaN,NaN,1
2,2014,18.0,Erkek,1/15/1986,Samsun,İstanbul,ULUSLARARASI KIBRIS ÜNİVERSİTESİ,Özel,100.0,Hayır,...,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,2
3,2014,40.0,Erkek,6/4/1991,Diyarbakır,İstanbul,İSTANBUL ŞEHİR ÜNİVERSİTESİ,Özel,100.0,Evet,...,0,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,3
4,2014,24.0,Erkek,2 Kasim 1992,Ankara/Altındağ,Ankara,TURGUT ÖZAL ÜNİVERSİTESİ,Özel,100.0,Evet,...,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,Erkek,18.06.2003 00:00,İstanbul,İstanbul,İstanbul Aydın Üniversitesi,Özel,NaN,Evet,...,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65120
65121,2022,0.0,Erkek,18.04.2002 00:00,Hatay,Muğla,Muğla Sıtkı Koçman Üniversitesi,Devlet,NaN,Hayır,...,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65121
65122,2022,0.0,Erkek,23.10.1999 00:00,Mersin,Mersin,Süleyman Demirel Üniversitesi,Devlet,NaN,Hayır,...,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65122
65123,2022,0.0,Erkek,17.11.2000 00:00,Samsun,Samsun,Karabük Üniversitesi,Devlet,NaN,Hayır,...,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65123


Understanding the data

In [ ]:
#yeni kod
old.info(), old.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65125 entries, 0 to 65124
Data columns (total 44 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   Basvuru Yili                                               65125 non-null  int64  
 1   Degerlendirme Puani                                        65124 non-null  float64
 2   Cinsiyet                                                   64956 non-null  object 
 3   Dogum Tarihi                                               64948 non-null  object 
 4   Dogum Yeri                                                 64334 non-null  object 
 5   Ikametgah Sehri                                            63088 non-null  object 
 6   Universite Adi                                             64993 non-null  object 
 7   Universite Turu                                            64870 non-null  object 
 8   Burslu

(None,
 Basvuru Yili                                                     0
 Degerlendirme Puani                                              1
 Cinsiyet                                                       169
 Dogum Tarihi                                                   177
 Dogum Yeri                                                     791
 Ikametgah Sehri                                               2037
 Universite Adi                                                 132
 Universite Turu                                                255
 Burslu ise Burs Yuzdesi                                      42685
 Burs Aliyor mu?                                                  0
 Bölüm                                                          231
 Universite Kacinci Sinif                                       374
 Universite Not Ortalamasi                                     2753
 Daha Once Baska Bir Universiteden Mezun Olmus                37345
 Lise Adi                                

In [ ]:
#yeni kod
old.nunique()

,0
Basvuru Yili,9
Degerlendirme Puani,97
Cinsiyet,4
Dogum Tarihi,13498
Dogum Yeri,2230
Ikametgah Sehri,319
Universite Adi,527
Universite Turu,4
Burslu ise Burs Yuzdesi,51
Burs Aliyor mu?,5


In [ ]:
#yeni kod
old[old.duplicated()]

,Basvuru Yili,Degerlendirme Puani,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,...,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",id


## Veri Türlerini ve Eksik Değerleri Kontrol Etme:

Bu kod, veri setinin özetini çıkarır. Sütunlardaki veri tiplerini, benzersiz değer sayısını, eksik veri miktarını ve eksik veri yüzdesini gösterir.


In [ ]:
#yeni kod
unique_values = old.nunique()
data_types = old.dtypes
missing_values = old.isnull().sum()
null_percentage = (old.isnull().sum() / len(old)) * 100
summary_df = pd.DataFrame({
    'Data Types': data_types,
    'Unique Values': unique_values,
    'Null Values': missing_values,
    "Null Percentage": null_percentage
});
summary_df


,Data Types,Unique Values,Null Values,Null Percentage
Basvuru Yili,int64,9,0,0.000000
Degerlendirme Puani,float64,97,1,0.001536
Cinsiyet,object,4,169,0.259501
Dogum Tarihi,object,13498,177,0.271785
Dogum Yeri,object,2230,791,1.214587
Ikametgah Sehri,object,319,2037,3.127831
Universite Adi,object,527,132,0.202687
Universite Turu,object,4,255,0.391555
Burslu ise Burs Yuzdesi,float64,51,42685,65.543186
Burs Aliyor mu?,object,5,0,0.000000


## Eksik Değerleri Değerlendirme Puanında Sıfır ile Doldurma:

Değerlendirme puanı eksik olan satırlarda, eksik olan değeri 0 ile doldurur.

In [ ]:
#yeni kod
old["Degerlendirme Puani"] = old["Degerlendirme Puani"].fillna(0)

In [ ]:
#yeni kod
missing_values_intrain = old.isnull().mean() * 100
print("Columns in df_train with more than 10% missing values:")
print(missing_values_intrain[missing_values_intrain > 10])

Columns in df_train with more than 10% missing values:
Burslu ise Burs Yuzdesi                                      65.543186
Daha Once Baska Bir Universiteden Mezun Olmus                57.343570
Lise Adi Diger                                               92.732438
Lise Bolum Diger                                             97.656814
Burs Aldigi Baska Kurum                                      67.527063
Baska Kurumdan Aldigi Burs Miktari                           67.811132
Anne Calisma Durumu                                          18.266411
Anne Sektor                                                  53.031862
Baba Calisma Durumu                                          19.375048
Baba Sektor                                                  31.290595
Uye Oldugunuz Kulubun Ismi                                   74.765451
Spor Dalindaki Rolunuz Nedir?                                49.558541
Aktif olarak bir STK üyesi misiniz?                          19.719002
Hangi STK'nin Uyesisin

## %95'ten Fazla Eksik Veriye Sahip Sütunları Kaldırma:

* Bu kod, %95'ten fazla eksik veriye sahip sütunları kaldırır. Bu sütunlar, verilerin çoğunun eksik olduğu ve analizde faydalı olmayacağı için çıkarılır.

* Ayrıca, "id" sütunu da çıkarılır çünkü genellikle bu sütun benzersiz kimlikler içerir ve makine öğrenmesi için anlamlı bir özellik değildir.


In [ ]:
#yeni kod
cols_drop = missing_values_intrain[missing_values_intrain > 95].index
old = old.drop(columns=cols_drop)
old.drop(["id"], axis=1, inplace=True)

## Sütun adlarındaki '?' karakterlerini boşluk ile değiştir:

In [ ]:
#yeni kod
old.columns = old.columns.str.replace('?', '');

* Çoğu makine öğrenmesi modeli sayılarla çalışır, bu yüzden OrdinalEncoder metinleri sayılara çevirir.
* Her sütunu şifreledikten sonra, test verilerinde de kullanabilmek için bu şifreleme modelini kaydederiz.

## Kategorik Değişkenlerin Ordinal Encoder ile Sayısal Değerlere Çevrilmesi

Bu bölümde, veriseti içerisindeki kategorik (object) değişkenleri **Ordinal Encoder** kullanarak sayısal verilere dönüştüreceğiz. Bu işlem, kategorik değişkenleri makine öğrenmesi algoritmalarıyla çalışabilecek hale getirmek için yapılır.

### Algoritmanın Adımları:

1. **Kategorik Değişkenlerin Seçimi:**
   - Verisetindeki tüm **object** tipindeki sütunlar seçilir.
   
2. **Ordinal Encoder ile Sayısal Değerlere Dönüştürme:**
   - Her bir kategorik sütun için **Ordinal Encoder** kullanılarak bu sütunlar sayısal değerlere dönüştürülür.
   - Bilinmeyen bir kategoriyle karşılaşıldığında bu değer `-1` olarak kodlanır.

3. **Encoder'ların Kaydedilmesi:**
   - Her sütun için oluşturulan encoder, `.joblib` uzantılı dosya olarak kaydedilir.
   - Bu dosyalar, ileride modelin yeniden eğitilmesi veya test edilmesi gerektiğinde kullanılabilir.

### Kod:


In [ ]:
#yeni kod
cols_train = old.select_dtypes(include = ["object"]).columns
for col in cols_train:
    ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    old[col] = ordinal_encoder.fit_transform(old[col].values.reshape(-1,1).astype(str))
    name = "ordinal_encoder_" + str(col) + ".joblib"
    joblib.dump(ordinal_encoder, name)

In [ ]:
#yeni kod
old

,Basvuru Yili,Degerlendirme Puani,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu,...,Uye Oldugunuz Kulubun Ismi,Profesyonel Bir Spor Daliyla Mesgul musunuz,Spor Dalindaki Rolunuz Nedir,Aktif olarak bir STK üyesi misiniz,Hangi STK'nin Uyesisiniz,Stk Projesine Katildiniz Mi,Girisimcilikle Ilgili Deneyiminiz Var Mi,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz,Ingilizce Biliyor musunuz,Ingilizce Seviyeniz
0,2014,52.0,2.0,10182.0,114.0,28.0,439.0,4.0,100.0,1.0,...,1401.0,0.0,3.0,1.0,8202.0,0.0,1.0,7675.0,2.0,3.0
1,2014,30.0,2.0,11028.0,2028.0,310.0,439.0,4.0,100.0,2.0,...,11398.0,1.0,1.0,1.0,8202.0,0.0,0.0,1360.0,2.0,3.0
2,2014,18.0,2.0,338.0,1174.0,310.0,387.0,4.0,100.0,2.0,...,8771.0,1.0,1.0,1.0,8202.0,1.0,1.0,7675.0,2.0,3.0
3,2014,40.0,2.0,11394.0,424.0,310.0,471.0,4.0,100.0,1.0,...,4421.0,1.0,1.0,1.0,8202.0,0.0,1.0,7675.0,2.0,3.0
4,2014,24.0,2.0,5980.0,143.0,28.0,371.0,4.0,100.0,1.0,...,11398.0,1.0,1.0,1.0,8202.0,1.0,1.0,7675.0,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,2.0,3780.0,2114.0,310.0,491.0,4.0,NaN,1.0,...,11398.0,1.0,0.0,1.0,10.0,2.0,1.0,14.0,1.0,3.0
65121,2022,0.0,2.0,3769.0,655.0,206.0,293.0,1.0,NaN,2.0,...,11398.0,1.0,0.0,1.0,10.0,2.0,1.0,14.0,1.0,3.0
65122,2022,0.0,2.0,7370.0,1025.0,202.0,354.0,1.0,NaN,2.0,...,11398.0,1.0,0.0,1.0,10.0,2.0,1.0,14.0,1.0,3.0
65123,2022,0.0,2.0,3585.0,1174.0,232.0,242.0,1.0,NaN,2.0,...,11398.0,1.0,0.0,1.0,10.0,2.0,1.0,14.0,1.0,3.0


Adding a new column for age

In [ ]:
old

,Degerlendirme Puani,Cinsiyet,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,Universite Kacinci Sinif,Universite Not Ortalamasi,Daha Once Baska Bir Universiteden Mezun Olmus,Lise Mezuniyet Notu,Baska Bir Kurumdan Burs Aliyor mu?,...,Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?,Profesyonel Bir Spor Daliyla Mesgul musunuz?,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,id,Yas
0,52.0,Erkek,Özel,100.0,Evet,3,3.50-3,Hayır,3.50-3,Hayır,...,Evet,Evet,Diğer,Hayır,Evet,Hayır,NaN,NaN,0,NaN
1,30.0,Erkek,Özel,100.0,Hayır,3,3.00-2.50,Hayır,3.00 - 4.00,Hayır,...,Hayır,Hayır,0,Hayır,Evet,Evet,NaN,NaN,1,NaN
2,18.0,Erkek,Özel,100.0,Hayır,1,NaN,Hayır,3.50-3,Hayır,...,Evet,Hayır,0,Hayır,Hayır,Hayır,NaN,NaN,2,NaN
3,40.0,Erkek,Özel,100.0,Evet,3,3.00-2.50,Hayır,3.50-3,Hayır,...,Evet,Hayır,0,Hayır,Evet,Hayır,NaN,NaN,3,NaN
4,24.0,Erkek,Özel,100.0,Evet,NaN,3.00-2.50,Hayır,3.00 - 4.00,Hayır,...,Hayır,Hayır,0,Hayır,Hayır,Hayır,NaN,NaN,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,0.0,Erkek,Özel,NaN,Evet,2,0 - 1.79,NaN,50 - 74,Hayır,...,Hayır,Hayır,-,Hayır,NaN,Hayır,Hayır,NaN,65120,NaN
65121,0.0,Erkek,Devlet,NaN,Hayır,1,Hazırlığım,NaN,75 - 100,Hayır,...,Hayır,Hayır,-,Hayır,NaN,Hayır,Hayır,NaN,65121,NaN
65122,0.0,Erkek,Devlet,NaN,Hayır,4,0 - 1.79,NaN,75 - 100,Evet,...,Hayır,Hayır,-,Hayır,NaN,Hayır,Hayır,NaN,65122,NaN
65123,0.0,Erkek,Devlet,NaN,Hayır,3,0 - 1.79,NaN,50 - 74,Hayır,...,Hayır,Hayır,-,Hayır,NaN,Hayır,Hayır,NaN,65123,NaN


Imputing

In [ ]:
#yeni kod
old.isnull().sum()

,0
Basvuru Yili,0
Degerlendirme Puani,0
Cinsiyet,0
Dogum Tarihi,0
Dogum Yeri,0
Ikametgah Sehri,0
Universite Adi,0
Universite Turu,0
Burslu ise Burs Yuzdesi,42685
Burs Aliyor mu,0


Encoding

In [ ]:
#yeni kod
old["Burslu ise Burs Yuzdesi"] = old["Burslu ise Burs Yuzdesi"].fillna(0)

In [ ]:
old

,Degerlendirme Puani,Cinsiyet,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,Universite Kacinci Sinif,Daha Once Baska Bir Universiteden Mezun Olmus,Baska Bir Kurumdan Burs Aliyor mu?,Baska Kurumdan Aldigi Burs Miktari,Anne Calisma Durumu,...,Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?,Profesyonel Bir Spor Daliyla Mesgul musunuz?,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,id,Yas
0,52.000000,2,1,100.000000,0,3,1,1,0,2,...,0,0,3,1,0,1,0,2,0,-21.158127
1,30.000000,2,1,100.000000,1,3,1,1,0,1,...,1,1,1,1,0,0,0,2,1,-21.158127
2,18.000000,2,1,100.000000,1,1,1,1,0,2,...,0,1,1,1,1,1,0,2,2,-21.158127
3,40.000000,2,1,100.000000,0,3,1,1,0,2,...,0,1,1,1,0,1,0,2,3,-21.158127
4,24.000000,2,1,100.000000,0,2,1,1,0,2,...,1,1,1,1,1,1,0,2,4,-21.158127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,0.000000,2,1,34.226181,0,2,1,1,0,2,...,1,1,0,1,1,1,1,2,65120,-21.158127
65121,0.000000,2,0,34.226181,1,1,1,1,0,2,...,1,1,0,1,1,1,1,2,65121,-21.158127
65122,0.000000,2,0,34.226181,1,4,1,0,384,2,...,1,1,0,1,1,1,1,2,65122,-21.158127
65123,0.000000,2,0,34.226181,1,3,1,1,0,1,...,1,1,0,1,1,1,1,2,65123,-21.158127


In [ ]:
X = old.drop(columns=["Degerlendirme Puani"])
y = old["Degerlendirme Puani"]

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
rfr.fit(X,y)

RandomForestRegressor()

# OLD VERİ SETİNİN EN SON HALİ:

In [ ]:
old


,Basvuru Yili,Degerlendirme Puani,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burs Aliyor mu,Bölüm,...,Uye Oldugunuz Kulubun Ismi,Profesyonel Bir Spor Daliyla Mesgul musunuz,Spor Dalindaki Rolunuz Nedir,Aktif olarak bir STK üyesi misiniz,Hangi STK'nin Uyesisiniz,Stk Projesine Katildiniz Mi,Girisimcilikle Ilgili Deneyiminiz Var Mi,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz,Ingilizce Biliyor musunuz,Ingilizce Seviyeniz
0,2014,52.0,2.0,10182.0,114.0,28.0,439.0,4.0,1.0,456.0,...,1401.0,0.0,3.0,1.0,8202.0,0.0,1.0,7675.0,2.0,3.0
1,2014,30.0,2.0,11028.0,2028.0,310.0,439.0,4.0,2.0,1247.0,...,11398.0,1.0,1.0,1.0,8202.0,0.0,0.0,1360.0,2.0,3.0
2,2014,18.0,2.0,338.0,1174.0,310.0,387.0,4.0,2.0,1773.0,...,8771.0,1.0,1.0,1.0,8202.0,1.0,1.0,7675.0,2.0,3.0
3,2014,40.0,2.0,11394.0,424.0,310.0,471.0,4.0,1.0,476.0,...,4421.0,1.0,1.0,1.0,8202.0,0.0,1.0,7675.0,2.0,3.0
4,2014,24.0,2.0,5980.0,143.0,28.0,371.0,4.0,1.0,4090.0,...,11398.0,1.0,1.0,1.0,8202.0,1.0,1.0,7675.0,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,2.0,3780.0,2114.0,310.0,491.0,4.0,1.0,3203.0,...,11398.0,1.0,0.0,1.0,10.0,2.0,1.0,14.0,1.0,3.0
65121,2022,0.0,2.0,3769.0,655.0,206.0,293.0,1.0,2.0,4614.0,...,11398.0,1.0,0.0,1.0,10.0,2.0,1.0,14.0,1.0,3.0
65122,2022,0.0,2.0,7370.0,1025.0,202.0,354.0,1.0,2.0,3470.0,...,11398.0,1.0,0.0,1.0,10.0,2.0,1.0,14.0,1.0,3.0
65123,2022,0.0,2.0,3585.0,1174.0,232.0,242.0,1.0,2.0,3883.0,...,11398.0,1.0,0.0,1.0,10.0,2.0,1.0,14.0,1.0,3.0


In [ ]:
old.keys()

Index(['Degerlendirme Puani', 'Cinsiyet', 'Universite Turu',
       'Burslu ise Burs Yuzdesi', 'Burs Aliyor mu?',
       'Universite Kacinci Sinif',
       'Daha Once Baska Bir Universiteden Mezun Olmus',
       'Baska Bir Kurumdan Burs Aliyor mu?',
       'Baska Kurumdan Aldigi Burs Miktari', 'Anne Calisma Durumu',
       'Anne Sektor', 'Baba Calisma Durumu', 'Baba Sektor', 'Kardes Sayisi',
       'Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?',
       'Profesyonel Bir Spor Daliyla Mesgul musunuz?',
       'Spor Dalindaki Rolunuz Nedir?', 'Aktif olarak bir STK üyesi misiniz?',
       'Stk Projesine Katildiniz Mi?',
       'Girisimcilikle Ilgili Deneyiminiz Var Mi?',
       'Ingilizce Biliyor musunuz?', 'Ingilizce Seviyeniz?', 'id', 'Yas'],
      dtype='object')

In [ ]:
old.keys()

Index(['Degerlendirme Puani', 'Cinsiyet', 'Universite Turu',
       'Burslu ise Burs Yuzdesi', 'Burs Aliyor mu?',
       'Universite Kacinci Sinif',
       'Daha Once Baska Bir Universiteden Mezun Olmus',
       'Baska Bir Kurumdan Burs Aliyor mu?',
       'Baska Kurumdan Aldigi Burs Miktari', 'Anne Calisma Durumu',
       'Anne Sektor', 'Baba Calisma Durumu', 'Baba Sektor', 'Kardes Sayisi',
       'Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?',
       'Profesyonel Bir Spor Daliyla Mesgul musunuz?',
       'Spor Dalindaki Rolunuz Nedir?', 'Aktif olarak bir STK üyesi misiniz?',
       'Stk Projesine Katildiniz Mi?',
       'Girisimcilikle Ilgili Deneyiminiz Var Mi?',
       'Ingilizce Biliyor musunuz?', 'Ingilizce Seviyeniz?', 'id', 'Yas'],
      dtype='object')

## Test Veri Seti Üzerinde Veri İşleme

In [ ]:
#yeni kod
new = pd.read_csv("./sample_data/test_x.csv")
new

,Basvuru Yili,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,Bölüm,...,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",id
0,2023,Kadın,21.06.2002 00:00,Amasya,Amasya,Munzur Üniversitesi,Devlet,NaN,Hayır,Sosyal Hizmet,...,Takım Oyuncusu,Evet,Kızılay da gönüllü dağıtım,NaN,Hayır,-,Evet,NaN,NaN,0
1,2023,Kadın,11.05.2004 00:00,Konya,Konya,Hacettepe Üniversitesi,Devlet,NaN,Hayır,Uluslararası İlişkiler,...,Bireysel Spor,Hayır,-,NaN,Hayır,-,Evet,NaN,NaN,1
2,2023,Erkek,24.04.2002 00:00,İstanbul,İstanbul,Kapadokya Üniversitesi,Özel,NaN,Evet,Diğer,...,Bireysel Spor,Hayır,-,NaN,Hayır,-,Evet,NaN,NaN,2
3,2023,Erkek,11.03.2003 00:00,Mardin,Mardin,Mardin Artuklu Üniversitesi,Devlet,NaN,Hayır,İslami İlimler,...,-,Evet,Lise gezilerde liderlik yapıyordum,NaN,Hayır,-,Hayır,NaN,NaN,3
4,2023,Erkek,26.01.2002 00:00,Samsun,İstanbul,Boğaziçi Üniversitesi,Devlet,NaN,Hayır,İşletme,...,-,Evet,"Toplum Gönüllüleri Vakfı'nda ""Yol Arkadaşım"" P...",NaN,Evet,"12 aydır, yolculuğuna bir yıl önce başlamış ol...",Evet,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023,Kadın,3.01.2002 00:00,Yozgat,Ankara,Gazi Üniversitesi,Devlet,NaN,Hayır,Endüstri Mühendisliği,...,-,Evet,TOG Vakfı'nda yaklaşık bir yıldır hem aktif gö...,NaN,Hayır,-,Evet,NaN,NaN,11044
11045,2023,Kadın,9.09.2001 00:00,Konya,Konya,Niğde Ömer Halisdemir Üniversitesi,Devlet,NaN,Hayır,Gıda Mühendisliği,...,-,Evet,Konyada düzenlenen 5. İslami dayanışma oyunlar...,NaN,Hayır,-,Hayır,NaN,NaN,11045
11046,2023,Erkek,8.06.2004 00:00,Ankara,Ankara,Karabük Üniversitesi,Devlet,NaN,Hayır,Elektrik-Elektronik Mühendisliği,...,Takım Oyuncusu,Evet,Hayvanlara yardım adı altında hayvanlara yardı...,NaN,Hayır,-,Evet,NaN,NaN,11046
11047,2023,Kadın,16.10.2001 00:00,Adıyaman,İstanbul,Bursa Uludağ Üniversitesi,Devlet,NaN,Hayır,"Moleküler Biyoloji, Genetik ve Biyomühendislik",...,Bireysel Spor,Evet,"Ağaç dikme, huzur evi ziyareti",NaN,Evet,Eğitim olarak deneyimim var. Aynı zamanda “Asp...,Evet,NaN,NaN,11047


In [ ]:
#yeni kod
new.drop(["Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite", "id", "Lise Bolum Diger"],axis = 1, inplace = True)

In [ ]:
#yeni kod
new.columns = new.columns.str.replace('?', '');

In [ ]:
#yeni kod
for col in cols_train:
    name = "ordinal_encoder_" + str(col) + ".joblib"
    l_e = joblib.load(name)
    new[col] = l_e.transform(new[col].values.reshape(-1,1).astype(str))

In [ ]:
#yeni kod
new

,Basvuru Yili,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu,Bölüm,...,Uye Oldugunuz Kulubun Ismi,Profesyonel Bir Spor Daliyla Mesgul musunuz,Spor Dalindaki Rolunuz Nedir,Aktif olarak bir STK üyesi misiniz,Hangi STK'nin Uyesisiniz,Stk Projesine Katildiniz Mi,Girisimcilikle Ilgili Deneyiminiz Var Mi,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz,Ingilizce Biliyor musunuz,Ingilizce Seviyeniz
0,2023,3.0,6936.0,128.0,27.0,291.0,1.0,NaN,2.0,4195.0,...,11398.0,0.0,8.0,0.0,-1.0,2.0,1.0,14.0,0.0,3.0
1,2023,3.0,-1.0,893.0,180.0,197.0,1.0,NaN,2.0,5069.0,...,11398.0,0.0,-1.0,1.0,10.0,2.0,1.0,14.0,0.0,3.0
2,2023,2.0,7547.0,2114.0,310.0,241.0,4.0,NaN,1.0,945.0,...,11398.0,1.0,-1.0,1.0,10.0,2.0,1.0,14.0,0.0,3.0
3,2023,2.0,1549.0,999.0,199.0,283.0,1.0,NaN,2.0,7430.0,...,11398.0,1.0,0.0,0.0,-1.0,2.0,1.0,14.0,1.0,3.0
4,2023,2.0,7958.0,1174.0,310.0,107.0,1.0,NaN,2.0,7552.0,...,11398.0,1.0,0.0,0.0,-1.0,2.0,0.0,-1.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023,3.0,8741.0,1396.0,28.0,176.0,1.0,NaN,2.0,1510.0,...,11398.0,1.0,0.0,0.0,-1.0,2.0,1.0,14.0,0.0,3.0
11045,2023,3.0,12951.0,893.0,180.0,305.0,1.0,NaN,2.0,2081.0,...,11398.0,1.0,0.0,0.0,-1.0,2.0,1.0,14.0,1.0,3.0
11046,2023,2.0,-1.0,135.0,28.0,242.0,1.0,NaN,2.0,1343.0,...,11398.0,0.0,8.0,0.0,-1.0,2.0,1.0,14.0,0.0,3.0
11047,2023,3.0,3366.0,70.0,310.0,112.0,1.0,NaN,2.0,3186.0,...,11398.0,0.0,-1.0,0.0,-1.0,2.0,0.0,-1.0,0.0,3.0


In [ ]:
#yeni kod
new["Burslu ise Burs Yuzdesi"] = new["Burslu ise Burs Yuzdesi"].fillna(0)

In [ ]:
#yeni kod
new.isnull().sum()

,0
Basvuru Yili,0
Cinsiyet,0
Dogum Tarihi,0
Dogum Yeri,0
Ikametgah Sehri,0
Universite Adi,0
Universite Turu,0
Burslu ise Burs Yuzdesi,0
Burs Aliyor mu,0
Bölüm,0


Encoding

In [ ]:
new

,Cinsiyet,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,Universite Kacinci Sinif,Daha Once Baska Bir Universiteden Mezun Olmus,Baska Bir Kurumdan Burs Aliyor mu?,Baska Kurumdan Aldigi Burs Miktari,Anne Calisma Durumu,Anne Sektor,...,Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?,Profesyonel Bir Spor Daliyla Mesgul musunuz?,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,id,Yas
0,2,0,NaN,1,0,NaN,1,0,0,3,...,1,0,4,0,NaN,1,0,NaN,0,NaN
1,2,0,NaN,1,0,NaN,1,0,1,0,...,1,0,1,1,NaN,1,0,NaN,1,NaN
2,1,1,NaN,0,0,NaN,0,2,1,0,...,1,1,1,1,NaN,1,0,NaN,2,NaN
3,1,0,NaN,1,0,NaN,1,0,1,0,...,1,1,0,0,NaN,1,1,NaN,3,NaN
4,1,0,NaN,1,2,NaN,0,3,0,2,...,1,1,0,0,NaN,0,0,NaN,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2,0,NaN,1,1,NaN,0,2,1,0,...,0,1,0,0,NaN,1,0,NaN,11044,NaN
11045,2,0,NaN,1,2,NaN,1,0,1,0,...,0,1,0,0,NaN,1,1,NaN,11045,NaN
11046,1,0,NaN,1,0,NaN,1,0,1,0,...,0,0,4,0,NaN,1,0,NaN,11046,NaN
11047,2,0,NaN,1,3,NaN,1,0,1,0,...,0,0,1,0,NaN,0,0,NaN,11047,NaN


## Veri setinin eğitim ve test kümelerine bölünmesi (train-test split)

In [ ]:
from sklearn.model_selection import train_test_split

#yeni kod
X = old.drop(columns=["Degerlendirme Puani"])
y = old["Degerlendirme Puani"]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

## Model Kıyaslaması

In [ ]:
#yeni kod

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, ElasticNet,
                                  BayesianRidge, Lars, LassoLars, HuberRegressor,
                                  OrthogonalMatchingPursuit)
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import (GradientBoostingRegressor, RandomForestRegressor,
                              ExtraTreesRegressor, AdaBoostRegressor)
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.metrics import mean_squared_error


# Modelleri tanımla
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Elastic Net': ElasticNet(),
    'Bayesian Ridge': BayesianRidge(),
    'Least Angle Regression': Lars(),
    'Lasso Least Angle Regression': LassoLars(),
    'Gradient Boosting Regressor': GradientBoostingRegressor(),
    'Random Forest Regressor': RandomForestRegressor(),
    'Extra Trees Regressor': ExtraTreesRegressor(),
    'AdaBoost Regressor': AdaBoostRegressor(),
    'Decision Tree Regressor': DecisionTreeRegressor(),
    'K Neighbors Regressor': KNeighborsRegressor(),
    'Huber Regressor': HuberRegressor(),
    'Orthogonal Matching Pursuit': OrthogonalMatchingPursuit(),
    'Dummy Regressor': DummyRegressor(strategy='mean'),
    'Hist Gradient Boosting Regressor': HistGradientBoostingRegressor()
}

results = {}

# Modelleri eğit ve değerlendir
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    results[name] = rmse
    print(f'{name}: RMSE = {rmse}')

# Sonuçları dataframe'e aktar ve sıralı olarak yazdır
results_df = pd.DataFrame(list(results.items()), columns=['Model', 'RMSE'])
print(results_df.sort_values(by='RMSE'))

Linear Regression: RMSE = 10.887173332999874
Ridge Regression: RMSE = 10.88719034405323
Lasso Regression: RMSE = 12.269095153714813
Elastic Net: RMSE = 13.524298719324207
Bayesian Ridge: RMSE = 10.887370450771625
Least Angle Regression: RMSE = 11.435539886987616
Lasso Least Angle Regression: RMSE = 12.269095287207538
Gradient Boosting Regressor: RMSE = 6.239865919606152
Random Forest Regressor: RMSE = 5.9422051716330255
Extra Trees Regressor: RMSE = 5.934628593875211
AdaBoost Regressor: RMSE = 10.406554349059874
Decision Tree Regressor: RMSE = 8.321057256029587
K Neighbors Regressor: RMSE = 14.596889607078928
Huber Regressor: RMSE = 16.8931116266912
Orthogonal Matching Pursuit: RMSE = 16.93843752368427
Dummy Regressor: RMSE = 18.241931761433612
Hist Gradient Boosting Regressor: RMSE = 5.676966993994213
                               Model       RMSE
16  Hist Gradient Boosting Regressor   5.676967
9              Extra Trees Regressor   5.934629
8            Random Forest Regressor   5.9

## En İyi Modeli Eğitme

In [ ]:
#yeni kod
HGBR = HistGradientBoostingRegressor()
HGBR.fit(X,y)

HistGradientBoostingRegressor()

##

## Model ile yeni veriler üzerinde tahmin yapma (prediction)

In [ ]:
#yeni kod
predhgbr = HGBR.predict(new)

In [ ]:
predhgbr

array([33.58225171, 20.48218435, 11.0659262 , ..., 37.18823768,
       48.97988966, 30.25824626])

In [ ]:
#yeni kod
new2 = pd.read_csv("./sample_data/test_x.csv")

In [ ]:
#yeni kod

df_submission = pd.DataFrame({'id': new2["id"], 'Degerlendirme Puani':predhgbr })

df_submission.to_csv('sonn.csv', index=False)

print("Predictions saved.")

Predictions saved.


In [ ]:
predict_hgbr = pd.DataFrame(predhgbr)

In [ ]:

predict_hgbr

,0
0,33.582252
1,20.482184
2,11.065926
3,18.087657
4,37.087314
...,...
11044,32.395460
11045,32.893986
11046,37.188238
11047,48.979890


In [ ]:
predict_hgbr.columns = ["Degerlendirme Puani"]

In [ ]:
predict_hgbr

,Degerlendirme Puani
0,33.582252
1,20.482184
2,11.065926
3,18.087657
4,37.087314
...,...
11044,32.395460
11045,32.893986
11046,37.188238
11047,48.979890


In [ ]:
predict_hgbr["id"] = new["id"]
predict_hgbr.set_index("id", inplace=True)

KeyError: "None of ['id'] are in the columns"

In [ ]:
predict_hgbr.to_csv("prediction_4.csv")